In [9]:
import sqlite3
import logging
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from transformers import pipeline
from typing import List, Dict, Tuple

# Configure logging to print to the notebook output
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Initialize the sentiment analysis pipelines
pipelines = {
    "FinBERT": pipeline("text-classification", model="ProsusAI/finbert"),
    "DistilBERT": pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", revision="af0f99b"),
    "FinBERT Tone": pipeline("text-classification", model="yiyanghkust/finbert-tone"),
    "DistilRoberta": pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"),
    "FinTwitBERT": pipeline("text-classification", model="StephanAkkerman/FinTwitBERT-sentiment"),
    "Sentiment Roberta": pipeline("text-classification", model="siebert/sentiment-roberta-large-english")
}

# Function to connect to the database and fetch news data
def fetch_news_from_db(db_path: str, query: str) -> List[Dict]:
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        # Convert rows to a list of dictionaries
        news_data = []
        for row in rows:
            news_data.append({"title": row[0], "text": row[1]})
        return news_data
    except sqlite3.Error as e:
        logger.error(f"Error fetching news data from database: {e}")
        return []

# Function to preprocess text data
def preprocess_text(text: str) -> str:
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Function to analyze sentiment using multiple models
def analyze_sentiment(text: str) -> List[Tuple[str, Dict]]:
    results = []
    text = preprocess_text(text)

    # VADER
    vader_result = sia.polarity_scores(text)
    if vader_result['compound'] != 0:
        label = 'POSITIVE' if vader_result['compound'] > 0 else 'NEGATIVE'
        results.append(('VADER', {'label': label, 'score': abs(vader_result['compound'])}))

    # Other models
    for model_name, sentiment_pipeline in pipelines.items():
        result = sentiment_pipeline(text)
        if result:
            label = result[0]['label']
            score = result[0]['score']
            # Normalize label and score interpretation
            if label.lower() in ['positive', 'negative', 'neutral', 'bullish', 'bearish']:
                normalized_label = label.upper()
                if normalized_label in ['POSITIVE', 'BULLISH']:
                    normalized_score = score
                elif normalized_label in ['NEGATIVE', 'BEARISH']:
                    normalized_score = -score
                else:
                    normalized_score = 0  # Neutral scores should contribute 0
                results.append((model_name, {'label': normalized_label, 'score': normalized_score}))

    return results

# Function to calculate the ensemble sentiment score
def ensemble_sentiment(sentiments: List[Tuple[str, Dict]]) -> Tuple[float, str]:
    weights = {
        'FinBERT': 3,
        'FinBERT Tone': 3,
        'Sentiment Roberta': 3,
        'DistilRoberta': 2,
        'FinTwitBERT': 1,
        'DistilBERT': 1,
        'VADER': 1
    }
    score = 0
    total_weight = 0
    for model, sentiment in sentiments:
        weight = weights.get(model, 1)
        total_weight += weight
        score += weight * sentiment['score']
    if total_weight == 0:
        return 0.0, "NEUTRAL"
    final_score = score / total_weight
    final_label = 'POSITIVE' if final_score > 0 else 'NEGATIVE'
    return final_score, final_label

# Function to process and display news articles
def process_news_articles(news_data: List[Dict]) -> None:
    if not news_data:
        logger.info("No news data to process.")
        return

    for article in news_data:
        headline = article.get("title", "")
        snippet = article.get("text", "")

        # Combine headline and snippet
        combined_text = f"{headline} {snippet}"

        # Analyze sentiment of the combined text
        combined_sentiments = analyze_sentiment(combined_text)

        # Calculate ensemble sentiment
        combined_ensemble_score, combined_ensemble_label = ensemble_sentiment(combined_sentiments)

        # Print the results
        print(f"\n{'-'*80}")
        print(f"Headline: {headline}")
        print(f"Snippet: {snippet}")
        print(f"Combined Ensemble Sentiment Score: {combined_ensemble_score:.2f} ({combined_ensemble_label})")
        print("Combined Sentiments:")
        for model, sentiment in combined_sentiments:
            print(f"  - {model}: {sentiment['label']} (score: {sentiment['score']:.2f})")
        print(f"{'-'*80}\n")

# Main function
def main():
    db_path = "News.db"
    query = "SELECT * FROM stock_news"

    news_data = fetch_news_from_db(db_path, query)
    if news_data:
        process_news_articles(news_data)
    else:
        logger.error("Failed to fetch news data.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--------------------------------------------------------------------------------
Headline: AMD
Snippet: AMD Inventory Headwinds & Robust MRVL Demand
Combined Ensemble Sentiment Score: 0.42 (POSITIVE)
Combined Sentiments:
  - VADER: POSITIVE (score: 0.23)
  - FinBERT: NEGATIVE (score: -0.73)
  - DistilBERT: POSITIVE (score: 0.98)
  - FinBERT Tone: POSITIVE (score: 1.00)
  - DistilRoberta: NEUTRAL (score: 0.00)
  - FinTwitBERT: BULLISH (score: 0.90)
  - Sentiment Roberta: POSITIVE (score: 1.00)
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Headline: GOOGL
Snippet: Google's Kurian approached Wiz, $23B deal could take a week to close, source says
Combined Ensemble Sentiment Score: -0.26 (NEGATIVE)
Combined Sentiments:
  - FinBERT: NEUTRAL (score: 0.00)
  - DistilBERT: NEGATIVE (score: -1.00)
  - FinBERT Tone: NEUTRAL (score: 0.00)
  - DistilRoberta: NEUTRAL (score: 0.00)
 

In [ ]:
import sqlite3

# Connect to the database [stock_news, fmp_articles]
conn = sqlite3.connect('News.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a query
cursor.execute("SELECT * FROM stock_news")

# Fetch the results
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
conn.close()

('AMD', 'AMD Inventory Headwinds & Robust MRVL Demand', '2024-07-09 12:05:35', 'youtube.com', "AMD's (AMD) takeaways from Asia are mixed, according to a Keybanc analyst. The analyst also highlights that Marvell's (MRVL) feedback indicates demand for optical networking is robust across all segments.", 'https://www.youtube.com/watch?v=3CG2Y41ueUg')
('GOOGL', "Google's Kurian approached Wiz, $23B deal could take a week to close, source says", '2024-07-15 14:34:47', 'techcrunch.com', "Alphabet, Google's parent company, is in advanced talks to acquire Wiz for $23 billion, a person close to the company told TechCrunch. The deal discussions were previously reported by the Wall Street Journal.", 'https://techcrunch.com/2024/07/15/googles-kurian-approached-wiz-23b-deal-could-take-a-week-to-close-source-says/')
('GOOG', 'Alphabet (GOOGL) Exceeds Market Returns: Some Facts to Consider', '2024-07-15 18:51:05', 'zacks.com', 'The latest trading day saw Alphabet (GOOGL) settling at $186.53, represent

In [ ]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url = ("https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page=0&apikey=kZ2IufkTebqBJoodhTojSFLZpBqhyKbO")
print(get_jsonparsed_data(url))

<ipython-input-31-3b68382d3442>:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'SPHR', 'publishedDate': '2024-07-25T20:18:00.000Z', 'title': "Ken Griffin's Citadel boosts stake in James Dolan's Sphere weeks after Steve Cohen's investment", 'image': 'https://cdn.snapi.dev/images/v1/6/b/entertainment-diversified2-2546667.jpg', 'site': 'nypost.com', 'text': "Citadel's stake in Sphere Entertainment rose from 1.6% to 5.3%, according to SEC filings released on Monday.", 'url': 'https://nypost.com/2024/07/25/business/citadel-boosts-stake-in-james-dolans-sphere-after-steve-cohen-investment/', 'sentiment': 'Positive', 'sentimentScore': 0.35}, {'symbol': 'BNO', 'publishedDate': '2024-07-25T20:17:58.000Z', 'title': 'Crude Oil Price Forecast: Reversal Day Signals Strength', 'image': 'https://cdn.snapi.dev/images/v1/q/g/oil28-2475005-2546665.jpg', 'site': 'fxempire.com', 'text': 'Signs of a bullish reversal in crude oil emerge as it bounces from 77.06, reclaiming key levels and targeting a breakout above 79.45.', 'url': 'https://www.fxempire.com/forecasts/article/

In [7]:
import sqlite3
import logging
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from transformers import pipeline
from typing import List, Dict, Tuple
import json
from urllib.request import urlopen
import certifi
import pandas as pd

# Function to connect to the database and fetch news data
def fetch_news_from_db(db_path: str, query: str) -> List[Dict]:
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        # Convert rows to a list of dictionaries
        news_data = []
        for row in rows:
            news_data.append({"title": row[0], "text": row[1]})
        return news_data
    except sqlite3.Error as e:
        logger.error(f"Error fetching news data from database: {e}")
        return []

# Function to fetch sentiment data from the API
def get_jsonparsed_data(url: str) -> List[Dict]:
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

db_path = "News.db"
query = "SELECT * FROM stock_news"

news_data = fetch_news_from_db(db_path, query)
api_url = "https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page=0&apikey=kZ2IufkTebqBJoodhTojSFLZpBqhyKbO"
api_sentiments = get_jsonparsed_data(api_url)
# Extract symbol and sentiment from the API data
symbol_sentiments = [(item['symbol'], item['sentiment']) for item in api_sentiments]

# Convert to DataFrame
df = pd.DataFrame(symbol_sentiments, columns=['symbol', 'sentiment'])

df


<ipython-input-7-667ddcf334b8>:32: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


,symbol,sentiment
0,AMED,Positive
1,NHYDY,Positive
2,CAKE,Positive
3,NABZY,Positive
4,PGRE,Positive
...,...,...
95,CVS,Negative
96,ENPH,Negative
97,ODD,Negative
98,RIVN,Negative


In [19]:
accuracy_dict = {}

In [49]:
import sqlite3
import logging
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from transformers import pipeline
from typing import List, Dict, Tuple
import pandas as pd
import json
import certifi
from urllib.request import urlopen
from sklearn.metrics import accuracy_score

# Configure logging to print to the notebook output
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Initialize the sentiment analysis pipelines
pipelines = {
    "FinBERT": pipeline("text-classification", model="ProsusAI/finbert"),
    "DistilBERT": pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", revision="af0f99b"),
    "FinBERT Tone": pipeline("text-classification", model="yiyanghkust/finbert-tone"),
    "DistilRoberta": pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"),
    "FinTwitBERT": pipeline("text-classification", model="StephanAkkerman/FinTwitBERT-sentiment"),
    "Sentiment Roberta": pipeline("text-classification", model="siebert/sentiment-roberta-large-english")
}

# Function to connect to the database and fetch news data
def fetch_news_from_db(db_path: str, query: str) -> List[Dict]:
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        # Convert rows to a list of dictionaries
        news_data = []
        for row in rows:
            news_data.append({"title": row[0], "text": row[1]})
        return news_data
    except sqlite3.Error as e:
        logger.error(f"Error fetching news data from database: {e}")
        return []

# Function to preprocess text data
def preprocess_text(text: str) -> str:
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Function to analyze sentiment using multiple models
def analyze_sentiment(text: str) -> List[Tuple[str, Dict]]:
    results = []
    text = preprocess_text(text)

    # VADER
    vader_result = sia.polarity_scores(text)
    if vader_result['compound'] != 0:
        label = 'POSITIVE' if vader_result['compound'] > 0 else 'NEGATIVE'
        results.append(('VADER', {'label': label, 'score': abs(vader_result['compound'])}))

    # Other models
    for model_name, sentiment_pipeline in pipelines.items():
        result = sentiment_pipeline(text)
        if result:
            label = result[0]['label']
            score = result[0]['score']
            # Normalize label and score interpretation
            if label.lower() in ['positive', 'negative', 'neutral', 'bullish', 'bearish']:
                normalized_label = label.upper()
                if normalized_label in ['POSITIVE', 'BULLISH']:
                    normalized_score = score
                elif normalized_label in ['NEGATIVE', 'BEARISH']:
                    normalized_score = -score
                else:
                    normalized_score = 0  # Neutral scores should contribute 0
                results.append((model_name, {'label': normalized_label, 'score': normalized_score}))

    return results

# Function to calculate the ensemble sentiment score
def ensemble_sentiment(sentiments: List[Tuple[str, Dict]]) -> Tuple[float, str]:
    weights = {
        'FinBERT': 4,
        'FinBERT Tone': 3,
        'Sentiment Roberta': 3,
        'DistilRoberta': 2,
        'FinTwitBERT': 2,
        'DistilBERT': 1,
        'VADER': 1
    }
    score = 0
    total_weight = 0
    for model, sentiment in sentiments:
        weight = weights.get(model, 1)
        total_weight += weight
        score += weight * sentiment['score']
    if total_weight == 0:
        return 0.0, "NEUTRAL"
    final_score = score / total_weight
    final_label = 'POSITIVE' if final_score > 0 else 'NEGATIVE'
    return final_score, final_label, weights

# Function to build a dictionary of symbols and their ensemble sentiment scores
def build_symbol_sentiment_dict(news_data: List[Dict]) -> Dict[str, float]:
    symbol_sentiment_dict = {}
    for article in news_data:
        headline = article.get("title", "")
        snippet = article.get("text", "")

        # Extract symbol from headline (assuming the symbol is mentioned in the headline)
        match = re.search(r'\b[A-Z]{1,5}\b', headline)
        if match:
            symbol = match.group(0)
            combined_text = f"{headline} {snippet}"
            combined_sentiments = analyze_sentiment(combined_text)
            combined_ensemble_score, _, weights = ensemble_sentiment(combined_sentiments)
            symbol_sentiment_dict[symbol] = combined_ensemble_score

    return symbol_sentiment_dict, weights

# Function to fetch sentiment data from the API
def get_jsonparsed_data(url: str) -> List[Dict]:
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

# Fetch news data
db_path = "News.db"
query = "SELECT * FROM stock_news"
news_data = fetch_news_from_db(db_path, query)

# Build the symbol sentiment dictionary
symbol_sentiment_dict, weights = build_symbol_sentiment_dict(news_data)

# Fetch API sentiments
api_url = "https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page=0&apikey=kZ2IufkTebqBJoodhTojSFLZpBqhyKbO"
api_sentiments = get_jsonparsed_data(api_url)

# Extract symbol and sentiment from the API data
symbol_sentiments = [(item['symbol'], item['sentiment']) for item in api_sentiments]

# Convert API data to DataFrame
df = pd.DataFrame(symbol_sentiments, columns=['symbol', 'sentiment'])

# Convert the symbol sentiment dictionary to a DataFrame
symbol_sentiment_df = pd.DataFrame(list(symbol_sentiment_dict.items()), columns=['symbol', 'ensemble_sentiment'])

# Merge the two DataFrames on the 'symbol' column
comparison_df = pd.merge(df, symbol_sentiment_df, on='symbol', how='outer')

# Assign labels to the ensemble sentiment scores
comparison_df['ensemble_label'] = comparison_df['ensemble_sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))
comparison_df = comparison_df.dropna()
accuracy = accuracy_score(comparison_df['sentiment'], comparison_df['ensemble_label'])

accuracy_dict[weights.values()] = accuracy

accuracy


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
<ipython-input-49-90b9f0185522>:136: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


0.23076923076923078

In [50]:
accuracy_dict

{dict_values([4, 3, 3, 2, 2, 1, 1]): 0.4444444444444444,
 dict_values([3, 3, 3, 2, 2, 1, 1]): 0.4444444444444444,
 dict_values([5, 4, 3, 3, 2, 2, 1]): 0.3076923076923077,
 dict_values([4, 4, 3, 3, 2, 2, 1]): 0.3076923076923077,
 dict_values([4, 4, 4, 3, 2, 2, 1]): 0.23076923076923078,
 dict_values([4, 3, 3, 2, 2, 2, 1]): 0.3076923076923077,
 dict_values([6, 5, 4, 3, 1, 2, 2]): 0.23076923076923078,
 dict_values([4, 4, 3, 2, 2, 1, 1]): 0.23076923076923078,
 dict_values([4, 3, 3, 2, 2, 2, 1]): 0.3076923076923077,
 dict_values([4, 3, 4, 2, 2, 1, 1]): 0.23076923076923078,
 dict_values([4, 2, 3, 2, 2, 1, 1]): 0.23076923076923078,
 dict_values([4, 3, 3, 2, 2, 3, 1]): 0.23076923076923078,
 dict_values([4, 3, 3, 2, 3, 1, 1]): 0.3076923076923077,
 dict_values([3.5, 3, 3, 2, 2, 1, 1]): 0.23076923076923078,
 dict_values([4, 3, 3, 2, 2, 1, 3]): 0.23076923076923078}

In [51]:
import sqlite3
import logging
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from transformers import pipeline
from typing import List, Dict, Tuple

# Configure logging to print to the notebook output
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Initialize the sentiment analysis pipelines
pipelines = {
    "FinBERT": pipeline("text-classification", model="ProsusAI/finbert"),
    "DistilBERT": pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", revision="af0f99b"),
    "FinBERT Tone": pipeline("text-classification", model="yiyanghkust/finbert-tone"),
    "DistilRoberta": pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"),
    "FinTwitBERT": pipeline("text-classification", model="StephanAkkerman/FinTwitBERT-sentiment"),
    "Sentiment Roberta": pipeline("text-classification", model="siebert/sentiment-roberta-large-english")
}

# Function to connect to the database and fetch news data
def fetch_news_from_db(db_path: str, query: str) -> List[Dict]:
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        # Convert rows to a list of dictionaries
        news_data = []
        for row in rows:
            news_data.append({"title": row[0], "text": row[1]})
        return news_data
    except sqlite3.Error as e:
        logger.error(f"Error fetching news data from database: {e}")
        return []

# Function to preprocess text data
def preprocess_text(text: str) -> str:
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Function to analyze sentiment using multiple models
def analyze_sentiment(text: str) -> List[Tuple[str, Dict]]:
    results = []
    text = preprocess_text(text)

    # VADER
    vader_result = sia.polarity_scores(text)
    if vader_result['compound'] != 0:
        label = 'POSITIVE' if vader_result['compound'] > 0 else 'NEGATIVE'
        results.append(('VADER', {'label': label, 'score': abs(vader_result['compound'])}))

    # Other models
    for model_name, sentiment_pipeline in pipelines.items():
        result = sentiment_pipeline(text)
        if result:
            label = result[0]['label']
            score = result[0]['score']
            # Normalize label and score interpretation
            if label.lower() in ['positive', 'negative', 'neutral', 'bullish', 'bearish']:
                normalized_label = label.upper()
                if normalized_label in ['POSITIVE', 'BULLISH']:
                    normalized_score = score
                elif normalized_label in ['NEGATIVE', 'BEARISH']:
                    normalized_score = -score
                else:
                    normalized_score = 0  # Neutral scores should contribute 0
                results.append((model_name, {'label': normalized_label, 'score': normalized_score}))

    return results

# Function to calculate the ensemble sentiment score
def ensemble_sentiment(sentiments: List[Tuple[str, Dict]]) -> Tuple[float, str]:
    weights = {
        'FinBERT': 4,
        'FinBERT Tone': 3,
        'Sentiment Roberta': 3,
        'DistilRoberta': 2,
        'FinTwitBERT': 1,
        'DistilBERT': 1,
        'VADER': 1
    }
    score = 0
    total_weight = 0
    for model, sentiment in sentiments:
        weight = weights.get(model, 1)
        total_weight += weight
        score += weight * sentiment['score']
    if total_weight == 0:
        return 0.0, "NEUTRAL"
    final_score = score / total_weight
    final_label = 'POSITIVE' if final_score > 0 else 'NEGATIVE'
    return final_score, final_label

# Function to process and display news articles
def process_news_articles(news_data: List[Dict]) -> None:
    if not news_data:
        logger.info("No news data to process.")
        return

    for article in news_data:
        headline = article.get("title", "")
        snippet = article.get("text", "")

        # Combine headline and snippet
        combined_text = f"{headline} {snippet}"

        # Analyze sentiment of the combined text
        combined_sentiments = analyze_sentiment(combined_text)

        # Calculate ensemble sentiment
        combined_ensemble_score, combined_ensemble_label = ensemble_sentiment(combined_sentiments)

        # Print the results
        print(f"\n{'-'*80}")
        print(f"Headline: {headline}")
        print(f"Snippet: {snippet}")
        print(f"Combined Ensemble Sentiment Score: {combined_ensemble_score:.2f} ({combined_ensemble_label})")
        print("Combined Sentiments:")
        for model, sentiment in combined_sentiments:
            print(f"  - {model}: {sentiment['label']} (score: {sentiment['score']:.2f})")
        print(f"{'-'*80}\n")

# Main function
def main():
    db_path = "News.db"
    query = "SELECT * FROM stock_news"

    news_data = fetch_news_from_db(db_path, query)
    if news_data:
        process_news_articles(news_data)
    else:
        logger.error("Failed to fetch news data.")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--------------------------------------------------------------------------------
Headline: AMD
Snippet: AMD Inventory Headwinds & Robust MRVL Demand
Combined Ensemble Sentiment Score: 0.35 (POSITIVE)
Combined Sentiments:
  - VADER: POSITIVE (score: 0.23)
  - FinBERT: NEGATIVE (score: -0.73)
  - DistilBERT: POSITIVE (score: 0.98)
  - FinBERT Tone: POSITIVE (score: 1.00)
  - DistilRoberta: NEUTRAL (score: 0.00)
  - FinTwitBERT: BULLISH (score: 0.90)
  - Sentiment Roberta: POSITIVE (score: 1.00)
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Headline: GOOGL
Snippet: Google's Kurian approached Wiz, $23B deal could take a week to close, source says
Combined Ensemble Sentiment Score: -0.25 (NEGATIVE)
Combined Sentiments:
  - FinBERT: NEUTRAL (score: 0.00)
  - DistilBERT: NEGATIVE (score: -1.00)
  - FinBERT Tone: NEUTRAL (score: 0.00)
  - DistilRoberta: NEUTRAL (score: 0.00)
 